In [1]:
import numpy as np
w_init = np.load('./w_1.npy')
b_init = np.load('./b_1.npy')
import tensorflow as tf

with tf.Session() as sess:
    # placeholder
    x = tf.placeholder(tf.float32, shape=[None, 1024], name="x")
    y = tf.placeholder(tf.float32, [None, 100], name="y")
    w_1 = tf.placeholder(tf.float32, [1024,1000], name="weights_in_1")
    w_2 = tf.placeholder(tf.float32, [1000,800], name="weights_in_2")
    w_3 = tf.placeholder(tf.float32, [800,100], name="weights_in_3")
    b_1 = tf.placeholder(tf.float32, [1000], name="bias_in_1")
    b_2 = tf.placeholder(tf.float32, [800], name="bias_in_2")
    b_3 = tf.placeholder(tf.float32, [100], name="bias_in_3")
    
    
    # 1st FC
    W_1 = tf.Variable(w_1, name="weights_1")
    tf.assign(W_1, w_init, name = "init_w_1")
    B_1 = tf.Variable(b_1, name="bias_1")
    tf.assign(B_1, b_init, name="init_b_1")
    out_1 = tf.matmul(x, W_1) + B_1
    #out_1 = tf.nn.relu(out_1)
    
    
    # 2nd FC
    W_2 = tf.Variable(w_2, name="weights_2")
    B_2 = tf.Variable(b_2, name="bias_2")
    out_2 = tf.add(tf.matmul(out_1, W_2), B_2, name="features")
    #out_2 = tf.nn.relu(out_2)
    
    # 3rd FC
    W_3 = tf.Variable(w_3, name="weights_3")
    B_3 = tf.Variable(b_3, name="bias_3")
    y_out = tf.add(tf.matmul(out_2, W_3), B_3)
    

    #cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_out), reduction_indices=[1]))
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_out))
    train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cross_entropy, name="train")

    correct_prediction = tf.equal(tf.argmax(y_out,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="test")

    init = tf.variables_initializer(tf.global_variables(), name="init")

    tf.train.write_graph(sess.graph_def,
                         './',
                         'mobilenet_FC_cifar100_D800.pb', as_text=False)